In [ ]:
#math library for writing sq.rt
import math

# General imports
import numpy as np

#AerSimulator
from qiskit_aer import AerSimulator

# Pre-defined ansatz circuit and operator class for Hamiltonian
#from qiskit.circuit.library import EfficientSU2
from qiskit.quantum_info import SparsePauliOp

# SciPy minimizer routine
from scipy.optimize import minimize

#for QAOA ansatz
from qiskit.circuit.library import QAOAAnsatz

# Plotting functions
import matplotlib.pyplot as plt

#importing the primitives and scipy optimizer
#from qiskit_ibm_runtime import Session, EstimatorV2 as Estimator
from scipy.optimize import minimize

import mthree
from iqm.qiskit_iqm import optimize_single_qubit_gates
from qiskit.result import sampled_expectation_value

In [ ]:
#taking the coeff.s of the Hamiltonian from Mathematica file OneDrive-清華大學/Chi-Kwong Li/TSP/Codes/mathematica/TSL/TSL_newP_5c_withR.nb
#Also see the overleaf file for reference : https://www.overleaf.com/project/6524d9a57849dc8791655795
d = 20

l00 = d
l01 =  math.sqrt(2)
l02 =  math.sqrt(10)
l03 = 3
l10 =  math.sqrt(2)
l11 = d
l12 = 2
l13 = math.sqrt(5)
l20 = math.sqrt(10)
l21 = 2
l22 = d
l23 = 1
l30 = 3
l31 = math.sqrt(5)
l32 = 1
l33 = d

In [ ]:
def tensor_diagonal(diagonal, identity_dim, mode='right'):
    """
    Compute the diagonal of a tensor product involving a diagonal matrix and an identity matrix.

    Parameters:
    - diagonal (array-like): The diagonal entries of the diagonal matrix.
    - identity_dim (int): The dimension of the identity matrix.
    - mode (str): Either 'right' for (diag ⊗ I) or 'left' for (I ⊗ diag).

    Returns:
    - np.ndarray: The diagonal elements of the tensor product.
    """
    diagonal = np.array(diagonal)

    if mode == 'right':
        # diag ⊗ I_n → repeat each element n times
        result = np.repeat(diagonal, identity_dim)
    elif mode == 'left':
        # I_n ⊗ diag → tile the diagonal n times
        result = np.tile(diagonal, identity_dim)
    else:
        raise ValueError("Invalid mode. Choose 'right' or 'left'.")
    
    return result

In [ ]:
D1 = [l30, l31, l32, d]
D1_vec = tensor_diagonal(D1, 2**4, mode='right')

C = [d, l01, l02, d, l10, d, l12, d, l20, l21, d, d, d, d, d, d]
C1_vec =  tensor_diagonal(C, 2**2, mode='right')

C2_vec =  tensor_diagonal(C, 2**2, mode='left')

D2 = [l03, l13, l23, d]
D2_vec = tensor_diagonal(D2, 2**4, mode='left')

In [ ]:
#Hamiltonian 
cost_ham_vec = D1_vec + C1_vec + C2_vec + D2_vec

In [ ]:
def extract_and_pad(arr, idx_to_extract, target_length, pad_value=40):
    """
    Extract elements by index and pad with a given value to reach target length.
    Raises an error if extracted length > target_length.

    Parameters:
    - arr: np.ndarray — original array
    - idx_to_extract: list of int — indices to extract
    - target_length: int — desired length of the output array
    - pad_value: any — value to pad the array with

    Returns:
    - np.ndarray — new array of length `target_length`
    """
    extracted = arr[idx_to_extract]
    current_length = len(extracted)

    padding = np.full(target_length - current_length, pad_value)
    return np.concatenate((extracted, padding))


In [ ]:
leg_travels = np.array([6, 9, 18, 24, 33, 36])
target_length = 2**3
pert_cost_ham_vec = extract_and_pad(cost_ham_vec, leg_travels, target_length, d)

In [ ]:
pert_cost_ham_vec

In [ ]:
import numpy as np
from qiskit.quantum_info import SparsePauliOp
from itertools import product

def generate_IZ_paulis(n):
    return [''.join(p) for p in product('IZ', repeat=n)]

def decompose_diagonal_to_pauli(diagonal):
    """
    Decomposes a diagonal matrix into a sum of I/Z Pauli tensor products.
    
    Parameters:
    - diagonal (array-like): Length-2^n diagonal of the matrix.

    Returns:
    - SparsePauliOp representing the diagonal matrix in the IZ basis.
    """
    n = int(np.log2(len(diagonal)))
    pauli_strings = generate_IZ_paulis(n)

    # Hadamard matrix on Z/IZ basis is equivalent to applying Hadamard on bits
    H = hadamard_matrix(len(diagonal))
    coeffs = H @ diagonal / len(diagonal)

    return SparsePauliOp(pauli_strings, coeffs=coeffs)

def hadamard_matrix(n):
    """Fast construction of n x n Hadamard matrix where n = 2^k"""
    assert (n & (n - 1)) == 0, "Length must be power of 2"
    H = np.array([[1]])
    while H.shape[0] < n:
        H = np.block([[H, H], [H, -H]])
    return H


In [ ]:
cost_hamiltonian = decompose_diagonal_to_pauli(pert_cost_ham_vec)

In [ ]:
from iqm.qiskit_iqm import IQMProvider
from iqm.qiskit_iqm.fake_backends.fake_garnet import IQMFakeGarnet
from qiskit.compiler import transpile

IQM_TOKEN = "####" # Paste your own token generated at https://resonance.meetiqm.com/ here

provider=IQMProvider(url="https://cocos.resonance.meetiqm.com/garnet", token = IQM_TOKEN)
backend = provider.get_backend()
#backend = IQMFakeGarnet()

In [ ]:
circuit = QAOAAnsatz(cost_operator=cost_hamiltonian, reps=1)
circuit.measure_all()

# Transpile to IQM Backend
candidate_circuit = transpile(circuit, backend, seed_transpiler=1,  optimization_level=3) # by fixing the transpiler seed we always get the same circuit even though the transpilation alogrithm has random element and can produce different circuits each time

In [ ]:
## Assign solution parameters to ansatz
qc = candidate_circuit.assign_parameters(np.array([2.32121258, 5.3033147]))
qc = optimize_single_qubit_gates(qc)

shots = 10_000
mit_shots = 10_000

job = backend.run(qc, shots = shots)
counts = job.result().get_counts()
#counts = Aer.get_backend('qasm_simulator').run(ansatz, shots = shots).result().get_counts()


In [ ]:
# Readout Error Mitigation (REM)
mapping = mthree.utils.final_measurement_mapping(qc)
m3_mitigator = mthree.M3Mitigation(backend)
m3_mitigator.cals_from_system(mapping)
mitigated_counts = m3_mitigator.apply_correction(counts, mapping)

In [ ]:
def to_bitstring(integer, num_bits):
    result = np.binary_repr(integer, width=num_bits)
    return [int(digit) for digit in result]

keys = list(mitigated_counts.keys())
values = list(mitigated_counts.values())

most_likely = keys[np.argmax(np.abs(values))]

print("Result bitstring:", most_likely)

In [ ]:
# Sort the dictionary items by values in descending order
sorted_items = sorted(mitigated_counts.items(), key=lambda x: x[1], reverse=True)
# Display the sorted key-value pairs
for key, value in sorted_items:
    print(f'{key}: {value}')